# Population Models

This notebook demonstrates how to work with population models. See `xforms Demo` for details on xforms.

In [1]:
from pathlib import Path

import nems.uri
import nems.xforms as xforms
import nems.recording as recording

[nems.configs.defaults INFO] Saving log messages to /tmp/nems\NEMS 2020-05-28 135123.log


## Configuration

In [2]:
# get the data and results paths
results_dir = nems.get_setting('NEMS_RESULTS_DIR')
signals_dir = nems.get_setting('NEMS_RECORDINGS_DIR')

In [3]:
# download some demo data
recording.get_demo_recordings(signals_dir, 'TAR010c.NAT.fs100.tgz')
datafile = Path(signals_dir) / 'TAR010c.NAT.fs100.tgz'

## Data Loading and Preprocessing

In [4]:
load_command = 'nems.demo.loaders.demo_loader'
exptid = 'TAR010c'
batch = 271
siteid = 'TAR010c-18-2'

In pop models, you can specify a variable number of outputs using `R`, and the model will determine the proper number of output channels.

In [5]:
modelspec_name = 'dlog-wc.18x3.g-fir.1x10x3-relu.3-wc.3xR-lvl.R'

## Generate the Modelspec

In [17]:
meta = {'siteid': siteid, 'batch': batch, 'modelname': modelspec_name,
        'recording': exptid}

xforms_init_context = {'siteid': siteid, 'batch': int(batch)}
xforms_init_context['keywordstring'] = modelspec_name
xforms_init_context['meta'] = meta
xforms_init_context['recording_uri_list'] = [str(datafile)]

In [18]:
xfspec = []

In [19]:
xfspec.append(['nems.xforms.init_context', xforms_init_context])
xfspec.append(['nems.xforms.load_recordings', {}])
xfspec.append(['nems.preprocessing.resp_to_pc',
              {'pc_source': 'psth', 'overwrite_resp': False,
               'pc_count': 2}])
xfspec.append(['nems.xforms.split_by_occurrence_counts',
               {'epoch_regex': '^STIM_'}])
xfspec.append(['nems.xforms.average_away_stim_occurrences', {}])

xfspec.append(['nems.xforms.init_from_keywords', {}])

In [20]:
xfspec.append(['nems.analysis.fit_pop_model.init_pop_pca', {'flip_pcs': True}])

xfspec.append(['nems.analysis.fit_pop_model.fit_population_iteratively',
               {'fitter': 'scipy_minimize', 'tolerances': [1e-4, 3e-5],
                'tol_iter': 50, 'fit_iter': 10}])

In [21]:
xfspec.append(['nems.xforms.predict', {}])
xfspec.append(['nems.analysis.api.standard_correlation', {},
               ['est', 'val', 'modelspec', 'rec'], ['modelspec']])
xfspec.append(['nems.xforms.plot_summary', {}])

## Run the Analysis

In [ ]:
ctx = {}
for xfa in xfspec:
    ctx = xforms.evaluate_step(xfa, ctx)